## General Workflow

### 1) Identifying Business Problem
### 2) Data Preprocessing
### 3) Exploratory Data Analysis (EDA)
### 4) Model Development & Evaluation
### 5) Model Interpretability

### 1) Identifying Business Problem

In this case, we're asked to develop a model for an organization that helps non-profit organizations to find donors. This model will be used to predict whether a donor will spend more than or equal to 30€, given that this donor receives a re-activation campaign. The model will be trained based on the previous campaign, and then validated with another data from other campaign.

The model is expected to have a good performance and high interpretability. In addition, we also have to emphasize why does implementing the model is better than random guessing campaign. In the end, the model will be used to predict potential customers for the upcoming campaign, and which customers are the most effective to reach.

### 2) Data Preprocessing

Our data consist of 5 csv files which includes the donor database, the amount of donated, and the selected donors during previous campaign

In [55]:
import sys
import subprocess
import importlib

packages = ['pandas', 'numpy','pathlib','os','glob', 'textblob']
[subprocess.check_call(['pip', 'install', pkg]) 
for pkg in packages if not importlib.util.find_spec(pkg)]

[]

In [56]:
import pandas as pd
import numpy as np

from pathlib import Path
import pathlib
import os
import glob
from numpy import where

from textblob import TextBlob

import datetime


In [23]:
# Change to working dir
os.chdir("DSC_Use_Case/Data")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'DSC_Use_Case/Data'

### Load Data

In [78]:
# Loop through data folder and add all df's in a dict
data_dict={}
for txt_file in glob.glob("*.csv"):
    #list filenames
    filename = txt_file
    #create python tablenames
    df_name = str.replace(str.replace(txt_file,".csv","_raw")," ","_")
    #read files
    df_value = pd.read_csv(filename,  delimiter=",")
    #add to dict
    data_dict[df_name] = df_value

In [79]:
# Split dict into indivisual dataframes
for var in data_dict.keys():
    exec("{} = data_dict['{}']".format(var, var))

In [80]:
data_dict.keys()

dict_keys(['campaigns_raw', 'donors_raw', 'gifts_raw', 'selection_campaign_6169_raw', 'selection_campaign_7244_raw'])

### Data exploration

### Donors Data

In [94]:
# Drop 1st Column
donors = donors_raw
donors['dateOfBirth'] = pd.to_datetime(donors['dateOfBirth'])

campaign6169 = datetime.datetime(2018, 9, 4)

donors['Age'] = donors['dateOfBirth'].apply(lambda row: campaign6169.year - row.year -((campaign6169.month, campaign6169.day) < (row.month, row.day) ))
# Change ANTWERP provinces to Antwerp

donors['province'] = donors['province'].apply(lambda row: 'Antwerp' if row == 'ANTWERP' else row)

donors.head()

,donorID,zipcode,province,region,gender,language,dateOfBirth,Age
0,100001,1861.0,Flemish Brabant,Flanders,F,NL,1967-12-24,50
1,100002,2260.0,Antwerp,Flanders,M,NL,1952-01-14,66
2,100003,1780.0,Flemish Brabant,Flanders,M,NL,1986-12-23,31
3,100004,1020.0,Brussels,Brussels,F,FR,1952-03-29,66
4,100005,1020.0,Brussels,Brussels,F,FR,1986-06-25,32


### Gifts Data

In [98]:
# Drop 1st Column
gifts = gifts_raw.iloc[: , 1:]
#cleaning gift
# Change date into datetime object
gifts['date'] = pd.to_datetime(gifts['date'])
#
gifts['amount'] = gifts['amount'].str.replace(',', '.')
#
gifts['amount'] = gifts['amount'].astype('float')

# Assume Independent Variable Period of 6 Months
iv_period = 180
#
in_between_period = 30

#first gift table 6169 campaign
#_______________________________________________
#
selection_campaign_6169=selection_campaign_6169_raw
# Only select rows within our Independent Variable Timeline
gifts_dv_6169 = gifts[gifts['campaignID'] == 6169]
# Aggregate Data for each donors
gifts_dv_6169 = gifts_dv_6169.groupby('donorID')['amount'].agg('sum')
# Reset Index and drop 2 level columns
gifts_dv_6169 = gifts_dv_6169.reset_index()
# Rename columns
gifts_dv_6169.columns = ['donorID','total_donated_during_dv']
#
campaign6169_date = datetime.datetime(2018, 9, 4)
#
end_period = campaign6169_date - datetime.timedelta(days = in_between_period)
#
start_period = end_period - datetime.timedelta(days = iv_period)
# Only select rows within our Independent Variable Timeline
gifts_6169 = gifts[(gifts['date'] >= start_period) &  (gifts['date'] <= end_period)]
#
date_agg = gifts_6169.groupby('donorID')['date'].agg('max')
# Aggregate Data for each donors
gifts_6169 = gifts_6169.groupby('donorID')['amount'].agg(['sum', 'count'])
# Reset Index and drop 2 level columns
gifts_6169 = gifts_6169.reset_index()
#gifts_cleaned.columns = gifts_cleaned.columns.droplevel(0)

# Merge with aggregated Date
gifts_6169 = pd.merge(gifts_6169, date_agg, how='left', on='donorID')
#Rename columns
gifts_6169.columns = ['donorID','total_donated','frequency_donor', 'last_date_dono']
#
gifts_6169['Recency_dono'] = end_period - gifts_6169['last_date_dono']
#
gifts_6169['Recency_dono'] = gifts_6169['Recency_dono'].dt.days
# Build Base Table Based On the 6169 Campaign Data
gifts_6169 = pd.merge(selection_campaign_6169,gifts_6169, how='left', on='donorID')
# # Merge with iv table
gifts_6169 = pd.merge(gifts_6169, gifts_dv_6169, how='left', on='donorID')
# Merge with donor database
gifts_6169 = pd.merge(gifts_6169, donors, how='left', on='donorID')
# #
gifts_6169['donated_more_than30_duringdv'] = gifts_6169['total_donated_during_dv'].apply(lambda row: 1 if row >= 30 else 0)
# #____________________________________________________
#first gift table 7244 campaign
#_______________________________________________
#
selection_campaign_7244=selection_campaign_7244_raw
# Only select rows within our Independent Variable Timeline
gifts_dv_7244 = gifts[gifts['campaignID'] == 7244]
# Aggregate Data for each donors
gifts_dv_7244 = gifts_dv_7244.groupby('donorID')['amount'].agg('sum')
# Reset Index and drop 2 level columns
gifts_dv_7244 = gifts_dv_7244.reset_index()
# Rename columns
gifts_dv_7244.columns = ['donorID','total_donated_during_dv']
#
campaign7244_date = datetime.datetime(2019, 6, 18)
#
end_period = campaign7244_date - datetime.timedelta(days = in_between_period)
#
start_period = end_period - datetime.timedelta(days = iv_period)
# Only select rows within our Independent Variable Timeline
gifts_7244 = gifts[(gifts['date'] >= start_period) &  (gifts['date'] <= end_period)]
#
date_agg = gifts_7244.groupby('donorID')['date'].agg('max')
# Aggregate Data for each donors
gifts_7244 = gifts_7244.groupby('donorID')['amount'].agg(['sum', 'count'])
# Reset Index and drop 2 level columns
gifts_7244 = gifts_7244.reset_index()
#gifts_cleaned.columns = gifts_cleaned.columns.droplevel(0)

# Merge with aggregated Date
gifts_7244 = pd.merge(gifts_7244, date_agg, how='left', on='donorID')
#Rename columns
gifts_7244.columns = ['donorID','total_donated','frequency_donor', 'last_date_dono']
#
gifts_7244['Recency_dono'] = end_period - gifts_7244['last_date_dono']
#
gifts_7244['Recency_dono'] = gifts_7244['Recency_dono'].dt.days
# Build Base Table Based On the 7244 Campaign Data
gifts_7244 = pd.merge(selection_campaign_7244,gifts_7244, how='left', on='donorID')
# # Merge with iv table
gifts_7244 = pd.merge(gifts_7244, gifts_dv_7244, how='left', on='donorID')
# Merge with donor database
gifts_7244 = pd.merge(gifts_7244, donors, how='left', on='donorID')
# #
gifts_7244['donated_more_than30_duringdv'] = gifts_7244['total_donated_during_dv'].apply(lambda row: 1 if row >= 30 else 0)



gifts_7244

,donorID,total_donated,frequency_donor,last_date_dono,Recency_dono,total_donated_during_dv,zipcode,province,region,gender,language,dateOfBirth,Age,donated_more_than30_duringdv
0,100001,NaN,NaN,NaT,NaN,NaN,1861.0,Flemish Brabant,Flanders,F,NL,1967-12-24,50,0
1,100002,NaN,NaN,NaT,NaN,NaN,2260.0,Antwerp,Flanders,M,NL,1952-01-14,66,0
2,100008,NaN,NaN,NaT,NaN,NaN,1780.0,Flemish Brabant,Flanders,M,FR,1937-08-30,81,0
3,100009,NaN,NaN,NaT,NaN,NaN,1020.0,Brussels,Brussels,M,FR,1932-01-20,86,0
4,100010,NaN,NaN,NaT,NaN,NaN,1020.0,Brussels,Brussels,M,FR,1983-05-30,35,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25640,144682,NaN,NaN,NaT,NaN,NaN,1640.0,Flemish Brabant,Flanders,M,FR,1982-08-29,36,0
25641,144683,NaN,NaN,NaT,NaN,NaN,9308.0,East Flanders,Flanders,NaN,NL,1985-06-15,33,0
25642,144686,NaN,NaN,NaT,NaN,NaN,2440.0,Antwerp,Flanders,F,NL,1964-09-28,53,0
25643,144690,NaN,NaN,NaT,NaN,NaN,1950.0,Flemish Brabant,Flanders,M,FR,1976-05-07,42,0


In [99]:
#create is_female variable
gifts_6169['is_female'] = where(gifts_6169["gender"]=='M', 1, 0)
#create is missing_gender variable
gifts_6169['is_missing_gender'] = where(gifts_6169["gender"]=='NaN', 1, 0)
#create missing_zipcode variable
gifts_6169['missing_zip'] = where(gifts_6169["zipcode"]=='NaN', 1, 0)
#create missing age variable
gifts_6169['missing_age'] = where(gifts_6169["Age"]=='NaN', 1, 0)
#create variable is Flanders
gifts_6169['is_flan'] = where(gifts_6169["region"]=='Flanders', 1, 0)
#create variable missing province
gifts_6169['missing_prov'] = where(gifts_6169["province"]=='NaN', 1, 0)
#create variable missing language
gifts_6169['missing_lang'] = where(gifts_6169["language"]=='NaN', 1, 0)
#create variables for languages
gifts_6169['speaks_en'] = where(gifts_6169["language"]=='EN', 1, 0)
gifts_6169['speaks_dutch'] = where(gifts_6169["language"]=='NL', 1, 0)
gifts_6169['speaks_fr'] = where(gifts_6169["language"]=='FR', 1, 0)
#create variable missing dob
gifts_6169['missing_dob'] = where(gifts_6169["dateOfBirth"]=='NaT', 1, 0)
#drop variables that arent necessary for the model
gifts_6169.pop('total_donated_during_dv')
gifts_6169.pop('province')
gifts_6169.pop('region')


gifts_6169['total_donated'] = gifts_6169['total_donated'].fillna((gifts_6169['total_donated'].mean()))

gifts_6169['Recency_dono'] = gifts_6169['Recency_dono'].fillna((gifts_6169['Recency_dono'].mean()))
gifts_6169['last_date_dono'] = gifts_6169['last_date_dono'].fillna((gifts_6169['last_date_dono'].median()))
gifts_6169['frequency_donor'] = gifts_6169['frequency_donor'].fillna((gifts_6169['frequency_donor'].median()))

yes_no_columns = ['gender']
for col in yes_no_columns:
    gifts_6169[col].replace({'F': 1,'M': 0},inplace=True)

In [101]:
gifts_6169

,donorID,total_donated,frequency_donor,last_date_dono,Recency_dono,zipcode,gender,language,dateOfBirth,Age,...,is_missing_gender,missing_zip,missing_age,is_flan,missing_prov,missing_lang,speaks_en,speaks_dutch,speaks_fr,missing_dob
0,100001,31.778418,1.0,2018-06-21,51.219388,1861.0,1.0,NL,1967-12-24,50.0,...,0,0,0,1,0,0,0,1,0,0
1,100002,31.778418,1.0,2018-06-21,51.219388,2260.0,0.0,NL,1952-01-14,66.0,...,0,0,0,1,0,0,0,1,0,0
2,100004,31.778418,1.0,2018-06-21,51.219388,1020.0,1.0,FR,1952-03-29,66.0,...,0,0,0,0,0,0,0,0,1,0
3,100008,31.778418,1.0,2018-06-21,51.219388,1780.0,0.0,FR,1937-08-30,81.0,...,0,0,0,1,0,0,0,0,1,0
4,100009,31.778418,1.0,2018-06-21,51.219388,1020.0,0.0,FR,1932-01-20,86.0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34883,144686,31.778418,1.0,2018-06-21,51.219388,2440.0,1.0,NL,1964-09-28,53.0,...,0,0,0,1,0,0,0,1,0,0
34884,144687,31.778418,1.0,2018-06-21,51.219388,4900.0,0.0,FR,1970-09-12,47.0,...,0,0,0,0,0,0,0,0,1,0
34885,144689,100.000000,1.0,2018-06-25,41.000000,3061.0,0.0,NL,1967-04-21,51.0,...,0,0,0,1,0,0,0,1,0,0
34886,144690,31.778418,1.0,2018-06-21,51.219388,1950.0,0.0,FR,1976-05-07,42.0,...,0,0,0,1,0,0,0,0,1,0


In [100]:
#create is_female variable
gifts_7244['is_female'] = where(gifts_7244["gender"]=='M', 1, 0)
#create is missing_gender variable
gifts_7244['is_missing_gender'] = where(gifts_7244["gender"]=='NaN', 1, 0)
#create missing_zipcode variable
gifts_7244['missing_zip'] = where(gifts_7244["zipcode"]=='NaN', 1, 0)
#create missing age variable
gifts_7244['missing_age'] = where(gifts_7244["Age"]=='NaN', 1, 0)
#create variable is Flanders
gifts_7244['is_flan'] = where(gifts_7244["region"]=='Flanders', 1, 0)
#create variable missing province
gifts_7244['missing_prov'] = where(gifts_7244["province"]=='NaN', 1, 0)
#create variable missing language
gifts_7244['missing_lang'] = where(gifts_7244["language"]=='NaN', 1, 0)
#create variables for languages
gifts_7244['speaks_en'] = where(gifts_7244["language"]=='EN', 1, 0)
gifts_7244['speaks_dutch'] = where(gifts_7244["language"]=='NL', 1, 0)
gifts_7244['speaks_fr'] = where(gifts_7244["language"]=='FR', 1, 0)
#create variable missing dob
gifts_7244['missing_dob'] = where(gifts_7244["dateOfBirth"]=='NaT', 1, 0)
#drop variables that arent necessary for the model
gifts_7244.pop('total_donated_during_dv')
gifts_7244.pop('province')
gifts_7244.pop('region')


gifts_7244['total_donated'] = gifts_7244['total_donated'].fillna((gifts_7244['total_donated'].mean()))

gifts_7244['Recency_dono'] = gifts_7244['Recency_dono'].fillna((gifts_7244['Recency_dono'].mean()))
gifts_7244['last_date_dono'] = gifts_7244['last_date_dono'].fillna((gifts_7244['last_date_dono'].median()))
gifts_7244['frequency_donor'] = gifts_7244['frequency_donor'].fillna((gifts_7244['frequency_donor'].median()))

yes_no_columns = ['gender']
for col in yes_no_columns:
    gifts_7244[col].replace({'F': 1,'M': 0},inplace=True)

In [102]:
gifts_7244

,donorID,total_donated,frequency_donor,last_date_dono,Recency_dono,zipcode,gender,language,dateOfBirth,Age,...,is_missing_gender,missing_zip,missing_age,is_flan,missing_prov,missing_lang,speaks_en,speaks_dutch,speaks_fr,missing_dob
0,100001,42.550312,1.0,2019-04-21,53.640625,1861.0,1.0,NL,1967-12-24,50,...,0,0,0,1,0,0,0,1,0,0
1,100002,42.550312,1.0,2019-04-21,53.640625,2260.0,0.0,NL,1952-01-14,66,...,0,0,0,1,0,0,0,1,0,0
2,100008,42.550312,1.0,2019-04-21,53.640625,1780.0,0.0,FR,1937-08-30,81,...,0,0,0,1,0,0,0,0,1,0
3,100009,42.550312,1.0,2019-04-21,53.640625,1020.0,0.0,FR,1932-01-20,86,...,0,0,0,0,0,0,0,0,1,0
4,100010,42.550312,1.0,2019-04-21,53.640625,1020.0,0.0,FR,1983-05-30,35,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25640,144682,42.550312,1.0,2019-04-21,53.640625,1640.0,0.0,FR,1982-08-29,36,...,0,0,0,1,0,0,0,0,1,0
25641,144683,42.550312,1.0,2019-04-21,53.640625,9308.0,NaN,NL,1985-06-15,33,...,0,0,0,1,0,0,0,1,0,0
25642,144686,42.550312,1.0,2019-04-21,53.640625,2440.0,1.0,NL,1964-09-28,53,...,0,0,0,1,0,0,0,1,0,0
25643,144690,42.550312,1.0,2019-04-21,53.640625,1950.0,0.0,FR,1976-05-07,42,...,0,0,0,1,0,0,0,0,1,0


In [34]:
# Assume Dependent Variable Period of 1 Month
dv_period = 30

campaign6169 = datetime.datetime(2018, 9, 4)
end_period = campaign6169 + datetime.timedelta(days = dv_period)

print(end_period)

2018-10-04 00:00:00


In [39]:
gifts_dv.groupby('donorID')['amount'].count().sort_values(ascending = False).head(10)

donorID
100023    1
129742    1
129665    1
129587    1
129512    1
129500    1
129493    1
129470    1
129395    1
129385    1
Name: amount, dtype: int64

In [46]:
# Example of Donors who were selected for a campaign, but we dont have his/her data in the donor database
selection_campaign_6169_raw[selection_campaign_6169_raw['donorID'] == 1474536]

,donorID
4361,1474536


### Base Table for the training set

Some info on this table:
- The basis table is the donors that were selected for campaign 6169
- IV period of 6 months, with gap period of 7 days
- DV period is not specified. Donors are selected on the basis of their reaction to the corresponding campaign (only takes into account campaign 6169)
- Granularity is DonorID

In [70]:
gifts_cleaned=gifts_cleaned.dropna()

/Users/inder/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [71]:
X=gifts_cleaned.drop(["donated_more_than30_duringdv",'last_date_dono','dateOfBirth','language'],axis=1)
y=gifts_cleaned['donated_more_than30_duringdv']

In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [73]:
lr = LogisticRegression()

In [74]:
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,stratify=y)

In [75]:
lr.fit(Xtrain,ytrain)

LogisticRegression()

In [76]:
predict_test=lr.predict(Xtest)
predict_train=lr.predict(Xtrain)

In [77]:
acc_train = accuracy_score(ytrain, np.round(predict_train))
acc_test = accuracy_score(ytest, np.round(predict_test))

print(f"Train:\tACC={acc_train:.4f}")
print(f"Test:\tACC={acc_test:.4f}")

Train:	ACC=0.9728
Test:	ACC=0.9727
